In [1]:
import torchvision.transforms as transforms

image_transform = transforms.Compose([
    transforms.Resize((448, 448)),
    transforms.ToTensor(),  # Converts image to (C, H, W)
])

from torchvision import transforms
from torchvision.transforms import InterpolationMode

mask_transform = transforms.Compose([
    transforms.Resize((448, 448), interpolation=InterpolationMode.NEAREST),  # Nearest-neighbor
    transforms.PILToTensor(),  # Converts to tensor without normalization
])

In [2]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
from natsort import natsorted  # For natural sorting (e.g., "image_10" comes after "image_9")

class TripletImageMaskDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, mask_transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.mask_transform = mask_transform
        
        # List and sort image/mask filenames
        self.image_files = natsorted([f for f in os.listdir(image_dir) if f.endswith('.jpg')])
        self.mask_files = natsorted([f for f in os.listdir(mask_dir) if f.endswith('.png')])
        
        # Ensure the number of images and masks match
        assert len(self.image_files) == len(self.mask_files), "Mismatch between images and masks!"
        
        # Exclude first and last images to form triplets (t-1, t, t+1)
        self.valid_indices = list(range(1, len(self.image_files)-1))

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        # Get the index for the middle image (t)
        t = self.valid_indices[idx]
        
        # Load triplet images: t-1, t, t+1
        img_t_minus1 = Image.open(os.path.join(self.image_dir, self.image_files[t-1])).convert("RGB")
        img_t = Image.open(os.path.join(self.image_dir, self.image_files[t])).convert("RGB")
        img_t_plus1 = Image.open(os.path.join(self.image_dir, self.image_files[t+1])).convert("RGB")
        
        # Load mask for the middle image (t)
        mask_t = Image.open(os.path.join(self.mask_dir, self.mask_files[t])).convert("L")  # Grayscale
        
        # Apply transformations
        if self.transform:
            img_t_minus1 = self.transform(img_t_minus1)
            img_t = self.transform(img_t)
            img_t_plus1 = self.transform(img_t_plus1)
            mask_t = self.mask_transform(mask_t)
        
        # Stack images along the batch dimension (B=3 for triplet)
        input_stack = torch.stack([img_t_minus1, img_t, img_t_plus1], dim=0)  # Shape: (3, C, H, W)

        print("Unique values in mask_t:", torch.unique(mask_t))

        return input_stack, mask_t


In [9]:
from torch.utils.data import random_split, Subset

# Initialize dataset
dataset = TripletImageMaskDataset(
    image_dir="Mask_Data/Images",
    mask_dir="Mask_Data/Masks",
    transform=image_transform,
    mask_transform=mask_transform
)
# Select only the first 10 instances
limited_dataset = Subset(dataset, range(10))

# Split into train (first 8) and validation (last 2)
train_dataset = Subset(limited_dataset, range(8))
val_dataset = Subset(limited_dataset, range(8, 10))

# Create DataLoaders
batch_size = 8  # Adjust batch size if needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

"""
# Split dataset into training and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
"""

'\n# Split dataset into training and validation\ntrain_size = int(0.8 * len(dataset))\nval_size = len(dataset) - train_size\ntrain_dataset, val_dataset = random_split(dataset, [train_size, val_size])\n\n# Create DataLoaders\nbatch_size = 8\ntrain_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)\nval_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)\n'

In [4]:
import torch
import torch.nn as nn

class CombinedLoss(nn.Module):
    def __init__(self, lambda1=0.4, lambda2=0.7):
        super(CombinedLoss, self).__init__()
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.bce = nn.BCEWithLogitsLoss()  # Binary Cross Entropy Loss

    def dice_loss(self, pred, target, smooth=1e-6):
        intersection = (pred * target).sum(dim=(2, 3))
        dice = (2. * intersection + smooth) / (pred.sum(dim=(2, 3)) + target.sum(dim=(2, 3)) + smooth)
        return 1 - dice.mean()  # Dice Loss (1 - Dice Coefficient)

    def forward(self, pred, target):
        # Ensure target is a float tensor
        target = target.float()  # Convert to float32
        bce_loss = self.bce(pred, target)
        
        # Apply sigmoid to pred for Dice loss
        pred_sigmoid = torch.sigmoid(pred)
        dice_loss = self.dice_loss(pred_sigmoid, target)
        
        return self.lambda2 * dice_loss + self.lambda1 * bce_loss

In [5]:
import torch.nn as nn
import torch.optim as optim
import FFSUnet

# Initialize model and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FFSUnet.FFS_UNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0004)
criterion = CombinedLoss()  # For binary masks

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0


    for batch_idx, (input_stack, mask_t) in enumerate(train_loader):
        input_stack = input_stack.to(device)  # Shape: (B, 3, C, H, W)
        mask_t = mask_t.to(device)            # Shape: (B, 1, H, W)

        # Forward pass
        outputs = model(input_stack)
        loss = criterion(outputs, mask_t)


        # Check gradients for a sample batch
        model.train()
        optimizer.zero_grad()
        outputs = model(input_stack)
        loss = criterion(outputs, mask_t)
        loss.backward()

        # Print gradients for key layers
        for name, param in model.named_parameters():
            if param.grad is not None:
                print(f"Gradient for {name}: {param.grad.abs().mean().item()}")
            else:
                print(f"No gradient for {name}!")

        epoch_loss += loss.item()

        # Print batch loss
        if (batch_idx + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.4f}")

/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import FFSUnet

# Initialize model and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FFSUnet.SimplifiedFFS_UNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0004)
criterion = CombinedLoss()  # For binary masks

# Get a single sample
#single_sample = dataset
#train_loader = DataLoader([single_sample], batch_size=8, shuffle=True)

# Track a specific weight (e.g., first convolutional layer)
def print_weights(model, epoch, prefix=""):
    for name, param in model.named_parameters():
        print(f"{prefix}Epoch {epoch+1} - {name}:")
        print(f"  Weight mean: {param.data.mean().item():.6f}")
        if param.grad is not None:
            print(f"  Gradient mean: {param.grad.mean().item():.6f}")
        else:
            print("  Gradient: None")

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0


    for batch_idx, (input_stack, mask_t) in enumerate(train_loader):
        input_stack = input_stack.to(device)  # Shape: (B, 3, C, H, W)
        mask_t = mask_t.to(device)            # Shape: (B, 1, H, W)

        # Check gradients for a sample batch
        model.train()
        optimizer.zero_grad()
        outputs = model(input_stack)
        loss = criterion(outputs, mask_t)
        loss.backward()

        # Print gradients for key layers
        for name, param in model.named_parameters():
            if param.grad is not None:
                print(f"Gradient for {name}: {param.grad.abs().mean().item()}")
            else:
                print(f"No gradient for {name}!")

        epoch_loss += loss.item()

        # Print batch loss
        if (batch_idx + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.4f}")

Unique values in mask_t: tensor([0, 1], dtype=torch.uint8)
Unique values in mask_t: tensor([0, 1], dtype=torch.uint8)
Unique values in mask_t: tensor([0, 1], dtype=torch.uint8)
Unique values in mask_t: tensor([0, 1], dtype=torch.uint8)
Unique values in mask_t: tensor([0, 1], dtype=torch.uint8)
Unique values in mask_t: tensor([0, 1], dtype=torch.uint8)
Unique values in mask_t: tensor([0, 1], dtype=torch.uint8)
Unique values in mask_t: tensor([0, 1], dtype=torch.uint8)
Gradient for fn_minus2.conv1.0.weight: 1.7298169041168876e-05
Gradient for fn_minus2.conv1.0.bias: 7.967552312360127e-11
Gradient for fn_minus2.conv1.1.weight: 5.435498678707518e-06
Gradient for fn_minus2.conv1.1.bias: 5.615085228782846e-06
Gradient for fn_minus2.stage2.0.weight: 3.1909357858239673e-06
Gradient for fn_minus2.stage2.0.bias: 2.582944058290848e-12
Gradient for fn_minus2.stage2.1.weight: 1.619170575395401e-06
Gradient for fn_minus2.stage2.1.bias: 1.4580357401428046e-06
Gradient for fn_minus2.stage2.3.weight: 2

KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import FFSUnet

# Assume CombinedLoss is defined/imported somewhere
# from your_loss_module import CombinedLoss

# Initialize model and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FFSUnet.FFS_UNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0004)
criterion = CombinedLoss()  # For binary masks

# Grab a single sample from the training loader
# This assumes that train_loader returns a tuple: (input_stack, mask_t)
single_sample = next(iter(train_loader))
input_sample, mask_sample = single_sample
input_sample = input_sample.to(device)  # Shape: (B, 3, C, H, W)
mask_sample = mask_sample.to(device)    # Shape: (B, 1, H, W)

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass using the single sample
    outputs = model(input_sample)
    loss = criterion(outputs, mask_sample)
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    # Print gradients for key layers (optional)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {loss.item():.4f}")


Unique values in mask_t: tensor([0, 1], dtype=torch.uint8)
Unique values in mask_t: tensor([0, 1], dtype=torch.uint8)
Unique values in mask_t: tensor([0, 1], dtype=torch.uint8)
Unique values in mask_t: tensor([0, 1], dtype=torch.uint8)
Unique values in mask_t: tensor([0, 1], dtype=torch.uint8)
Unique values in mask_t: tensor([0, 1], dtype=torch.uint8)
Unique values in mask_t: tensor([0, 1], dtype=torch.uint8)
Unique values in mask_t: tensor([0, 1], dtype=torch.uint8)
Epoch 1/50 - Loss: 0.9394
Epoch 2/50 - Loss: 0.8875
Epoch 3/50 - Loss: 0.7402
Epoch 4/50 - Loss: 0.7103
Epoch 5/50 - Loss: 0.7015
Epoch 6/50 - Loss: 0.6353
Epoch 7/50 - Loss: 0.5807


KeyboardInterrupt: 